# Anime Expo rankings

Hi! This is an overview of the main filters of Mangaki (top(), controversial(), random()), and how they could and should be improved.

It was presented at the **Anime & Manga Studies Symposium of Anime Expo** on July 2, 2017.

In [44]:
from collections import Counter

nb_likes = Counter()
nb_dislikes = Counter()
for rating in Rating.objects.exclude(choice__in=['willsee', 'wontsee']):
    if rating.choice == 'like':
        nb_likes[rating.work_id] += 1
    elif rating.choice ==  'favorite':
        nb_likes[rating.work_id] += 2
    elif rating.choice == 'neutral':
        nb_dislikes[rating.work_id] += 0.5
    elif rating.choice == 'dislike':
        nb_dislikes[rating.work_id] += 1

In [45]:
def f(x):
    return x
interact(f, x=5)

<function __main__.f>

In [28]:
from ipywidgets import interact, interactive
from IPython.display import display

In [51]:
def perles(max_ratings=126, min_ratings=20):
    ranking = Counter()
    for work_id in nb_likes:
        if min_ratings <= nb_likes[work_id] + nb_dislikes[work_id] <= max_ratings:
            dislike_rate = nb_dislikes[work_id] / nb_likes[work_id]
            ranking[work_id] = -dislike_rate
    rank = 1
    lines = []
    for work_id, dislike_rate in ranking.most_common(10):
        lines.append('%d. %s %d likes %d likes' % (rank, Work.objects.get(id=work_id).title, nb_likes[work_id], nb_dislikes[work_id]))
        rank += 1
    print('\n'.join(lines))

In [52]:
w = interactive(perles)

In [53]:
display(w)

In [1]:
# LaTeX output using the booktabs package

def header(cols):
    print(r'\begin{tabular}{%s} \toprule' % ('c' * len(cols)))
    print(' & '.join(cols) + r'\\ \midrule')

def footer():
    print(r'\end{tabular}')

In [3]:
header(['\#', 'Work', '\# Ratings'])
MAX = 10
for rank, work in enumerate(Work.objects.popular()[:MAX], start=1):
    print(r'{} & {} & {}\\{}'.format(rank, work, work.nb_ratings, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccc} \toprule
\# & Work & \# Ratings\\ \midrule
1 & Death Note & 1464\\
2 & L'Attaque des Titans & 1344\\
3 & Le Voyage de Chihiro & 1209\\
4 & Naruto & 1177\\
5 & Princesse Mononoké & 1170\\
6 & Sword Art Online & 1151\\
7 & Fullmetal Alchemist: Brotherhood & 1150\\
8 & Fullmetal Alchemist & 1032\\
9 & Naruto: Shippuuden & 996\\
10 & Fairy Tail & 985\\ \bottomrule
\end{tabular}


In [4]:
from django.db.models import F, FloatField, ExpressionWrapper
from django.db.models.functions import Cast
from collections import Counter

def get_ratings(work):
    return Counter(work.rating_set.values_list('choice', flat=True))

'''header(['\#', 'Work', ''])
MAX = 10
for rank, work in enumerate(Work.objects.top().reverse().annotate(score=ExpressionWrapper(F('nb_likes') / F('nb_ratings'), output_field=FloatField()))[:MAX], start=1):
    print(r'{} & {} {}\\{}'.format(rank, work, get_ratings(work), r' \bottomrule' if rank == MAX else ''))
footer()'''

"header(['\\#', 'Work', ''])\nMAX = 10\nfor rank, work in enumerate(Work.objects.top().reverse().annotate(score=ExpressionWrapper(F('nb_likes') / F('nb_ratings'), output_field=FloatField()))[:MAX], start=1):\n    print(r'{} & {} {}\\{}'.format(rank, work, get_ratings(work), r' \x08ottomrule' if rank == MAX else ''))\nfooter()"

In [5]:
from collections import defaultdict

ratings = defaultdict(Counter)
for _id, title, choice in Rating.objects.values_list('work_id', 'work__title', 'choice'):
    ratings[(_id, title)][choice] += 1

In [6]:
for key in ratings.copy():
    if sum(ratings[key].values()) < 10:
        del ratings[key]

In [7]:
sum(ratings[(1, 'Death Note')].values())

1623

In [8]:
def nb_seen_ratings(key):
    return ratings[key]['like'] + ratings[key]['favorite'] + ratings[key]['dislike']

In [9]:
header(['\#', 'Work', '\# Fav', '\# Like', '\# Hate'])
MAX = 20
for rank, key in enumerate(sorted(ratings.keys(), key=lambda key:
        float('inf') if nb_seen_ratings(key) < 50 else
       (float(4*ratings[key]['favorite'] + 2*ratings[key]['like'] + (-0.1)*ratings[key]['neutral'] + (-2)*ratings[key]['dislike'])
        / sum(ratings[key].values())), reverse=False)[:MAX], start=1):
    # print(key, ratings[key]['dislike'])
    print(r'{} & {} & {} & {}\\{}'.format(rank, key[1], ratings[key]['favorite'] + ratings[key]['like'], ratings[key]['dislike'], r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccccc} \toprule
\# & Work & \# Fav & \# Like & \# Hate\\ \midrule
1 & Dragon Ball GT & 97 & 161\\
2 & Glasslip & 24 & 36\\
3 & Girls Bravo: First Season & 26 & 39\\
4 & Captain Earth & 25 & 35\\
5 & Ai non stop! & 25 & 33\\
6 & Yu-Gi-Oh! GX & 128 & 158\\
7 & Choujigen Game Neptune: The Animation & 27 & 33\\
8 & Astarotte's Toy & 32 & 38\\
9 & Dog Days & 57 & 67\\
10 & Haruka Nogizaka's Secret & 24 & 27\\
11 & Wizard Barristers & 51 & 59\\
12 & Sakura Trick & 49 & 51\\
13 & .Hack - Le bracelet du crépuscule & 25 & 26\\
14 & Ikki Tousen & 35 & 39\\
15 & To Love & 56 & 57\\
16 & Rail Wars! & 61 & 57\\
17 & The Severing Crime Edge & 33 & 29\\
18 & Chaos;Head & 77 & 66\\
19 & Phi Brain: Puzzle of God & 30 & 23\\
20 & Freezing & 80 & 67\\ \bottomrule
\end{tabular}


In [10]:
header(['\#', 'Work', '# Like', '# Hate'])
MAX = 10
for rank, work in enumerate(Work.objects.controversial()[:MAX], start=1):
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {} & {}\\{}'.format(rank, work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{cccc} \toprule
\# & Work & # Like & # Hate\\ \midrule
1 & School Days & 182 & 158\\
2 & Dragon Ball Z Movie 11: Bio-Broly & 122 & 108\\
3 & Rail Wars! & 59 & 57\\
4 & Freezing & 74 & 67\\
5 & Chaos;Head & 75 & 66\\
6 & To Love & 52 & 56\\
7 & Kiss x Sis & 122 & 97\\
8 & Naruto the Movie 2: Legend of the Stone of Gelel & 114 & 90\\
9 & Yu-Gi-Oh! GX & 118 & 158\\
10 & Sakura Trick & 47 & 51\\ \bottomrule
\end{tabular}


In [11]:
header(['\#', 'Work', '# Like', '# Hate'])
MAX = 20
for rank, work in enumerate(Work.objects.random().order_by('?')[:MAX], start=1):
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {} & {}\\{}'.format(rank, work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{cccc} \toprule
\# & Work & # Like & # Hate\\ \midrule
1 & Dragon Ball Z Movie 12: Fusion Reborn & 43 & 9\\
2 & Saint Seiya & 81 & 13\\
3 & KINMOZA! & 17 & 5\\
4 & Seto no Hanayome OVA & 25 & 2\\
5 & Girls & Panzer & 29 & 9\\
6 & Neon Genesis Evangelion: Death & Rebirth & 35 & 10\\
7 & Pokemon Diamond & Pearl & 19 & 4\\
8 & Kaichou wa Maid-sama!: Goshujin-sama to Asonjao♥ & 30 & 2\\
9 & The Testament of Sister New Devil & 18 & 6\\
10 & The Tale of The Princess Kaguya & 171 & 14\\
11 & Final Fantasy VII: Last Order & 18 & 5\\
12 & Mushishi Zoku Shou Special & 42 & 2\\
13 & Bartender & 18 & 5\\
14 & Suisei no Gargantia Specials & 35 & 1\\
15 & Goodbye Mr. Despair OAD & 29 & 2\\
16 & Full Metal Panic! The Second Raid & 62 & 6\\
17 & Looking Up At The Half-Moon & 15 & 1\\
18 & The Arms Peddler & 61 & 5\\
19 & Beelzebub & autres histoires maléfiques & 44 & 8\\
20 & Steins;Gate: Fuka Ryouiki no Déjà vu & 166 & 7\\ \bottomrule
\end{tabular}


In [12]:
ratings[(3236, 'Kaiba')]

Counter({'dislike': 4,
         'favorite': 8,
         'like': 34,
         'neutral': 12,
         'willsee': 27,
         'wontsee': 41})

In [13]:
kaiba = Work.objects.get(title='Kaiba')
madlax = Work.objects.get(title='Madlax')
fujiko = Work.objects.filter(title__icontains='Fujiko')[3]
momo = Work.objects.get(id=902)

In [14]:
header(['Work', '# Like', '# Hate'])
MAX = 20
for work in [kaiba, madlax, fujiko, momo]:
    # print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')
    print(r'{} & {} & {}\\{}'.format(work.title, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\begin{tabular}{ccc} \toprule
Work & # Like & # Hate\\ \midrule
Kaiba & 34 & 4\\ \bottomrule
Madlax & 19 & 4\\ \bottomrule
Lupin the Third, The Woman Called Fujiko Mine & 14 & 1\\ \bottomrule
A Letter to Momo & 42 & 1\\ \bottomrule
\end{tabular}


In [15]:
get_ratings(fujiko)

Counter({'dislike': 1, 'like': 14, 'neutral': 5, 'willsee': 5, 'wontsee': 4})

In [16]:
get_ratings(momo)

Counter({'dislike': 1,
         'favorite': 2,
         'like': 42,
         'neutral': 13,
         'willsee': 31,
         'wontsee': 18})

In [17]:
get_ratings(kaiba)

Counter({'dislike': 4,
         'favorite': 8,
         'like': 34,
         'neutral': 12,
         'willsee': 27,
         'wontsee': 41})

In [18]:
get_ratings(Work.objects.get(title='Madlax'))

Counter({'dislike': 4,
         'favorite': 2,
         'like': 19,
         'neutral': 8,
         'willsee': 8,
         'wontsee': 18})

In [19]:
Work.objects.random().count()

896

In [20]:
Work.objects.filter(nb_dislikes__lte=1, nb_likes__gte=10).values_list('title')

<WorkQuerySet [('Spice and Wolf II Specials',), ('The Story of Saiunkoku',), ('Amagami SS OVA',), ('Kara no Kyoukai 6: Boukyaku Rokuon',), ('Noragami Aragoto',), ('Ponyo',), ('Danna ga Nani wo Itteiru ka Wakaranai Ken 2 Sure-me',), ('Cardcaptor Sakura: Leave It to Kero-chan',), ('Kara no Kyoukai: Mirai Fukuin - Extra Chorus',), ("Natsume's Book of Friends Season 2",), ('The Seven Metamorphoses of Yamato Nadeshiko',), ('Owari no Seraph: Owaranai Seraph',), ('Tower of Druaga: The Sword of Uruk',), ('Eureka Seven - good night, sleep tight, young lovers',), ('Honey and Clover II',), ('Fate/Prototype',), ('Danna ga Nani wo Itteiru ka Wakaranai Ken 2nd Season',), ('A Letter to Momo',), ('Psychic School Wars',), ('Non Non Biyori Repeat',), '...(remaining elements truncated)...']>

In [21]:
queryset = (Work.objects.exclude(nb_likes=0).annotate(
    dislike_rate=ExpressionWrapper(Cast(F('nb_dislikes'), FloatField()) / F('nb_likes'), output_field=FloatField()))
    .filter(nb_ratings__gte=30, nb_ratings__lte=126, dislike_rate__lte=5/42)
    .order_by('dislike_rate'))
print(r'\section{{{} perles rares}}'.format(queryset.count()))
header(['\#', 'Work', '# Ratings', '# Like', '# Hate'])
for rank, work in enumerate(queryset, start=1):
    print(r'{} & \href{{https://mangaki.fr/{}/{}}}{{{}}} & {} & {} & {}\\{}'.format(rank, work.category.slug, work.id, work.title, work.nb_ratings, work.nb_likes, work.nb_dislikes, r' \bottomrule' if rank == MAX else ''))
footer()

\section{228 perles rares}
\begin{tabular}{ccccc} \toprule
\# & Work & # Ratings & # Like & # Hate\\ \midrule
1 & \href{https://mangaki.fr/anime/9895}{Psychic School Wars} & 32 & 15 & 0\\
2 & \href{https://mangaki.fr/anime/9038}{GARO THE ANIMATION} & 31 & 22 & 0\\
3 & \href{https://mangaki.fr/anime/9071}{Psycho-Pass: The Movie} & 51 & 37 & 0\\
4 & \href{https://mangaki.fr/anime/2767}{The Boy and the Beast} & 60 & 30 & 0\\
5 & \href{https://mangaki.fr/anime/1380}{Kara no Kyoukai 6: Boukyaku Rokuon} & 82 & 54 & 1\\
6 & \href{https://mangaki.fr/anime/1304}{Natsume's Book of Friends Season 2} & 62 & 46 & 1\\
7 & \href{https://mangaki.fr/anime/9814}{Samurai X: Trust and Betrayal} & 58 & 45 & 1\\
8 & \href{https://mangaki.fr/anime/902}{A Letter to Momo} & 58 & 42 & 1\\
9 & \href{https://mangaki.fr/anime/9863}{True Tears} & 60 & 41 & 1\\
10 & \href{https://mangaki.fr/anime/10042}{My Teen Romantic Comedy SNAFU TOO!} & 48 & 41 & 1\\
11 & \href{https://mangaki.fr/anime/2913}{Tasogare Otome x Amn

183 & \href{https://mangaki.fr/anime/2256}{The World God Only Knows: Flag 0} & 39 & 20 & 2\\
184 & \href{https://mangaki.fr/anime/3868}{Kino's Journey} & 54 & 40 & 4\\
185 & \href{https://mangaki.fr/anime/9513}{Love Live! School Idol Project 2nd Season} & 55 & 39 & 4\\
186 & \href{https://mangaki.fr/anime/1899}{Love Lab} & 45 & 29 & 3\\
187 & \href{https://mangaki.fr/anime/1948}{Chihayafuru 2} & 120 & 76 & 8\\
188 & \href{https://mangaki.fr/anime/10375}{Tales of Vesperia: The First Strike} & 30 & 19 & 2\\
189 & \href{https://mangaki.fr/anime/8409}{Hunter x Hunter OVA} & 105 & 75 & 8\\
190 & \href{https://mangaki.fr/manga/6655}{Noragami} & 91 & 56 & 6\\
191 & \href{https://mangaki.fr/anime/9587}{Tari Tari} & 49 & 28 & 3\\
192 & \href{https://mangaki.fr/anime/1636}{Sekaiichi Hatsukoi 2} & 38 & 28 & 3\\
193 & \href{https://mangaki.fr/anime/14510}{Toaru Kagaku no Railgun: Misaka-san wa Ima Chuumoku no Mato Desukara} & 55 & 37 & 4\\
194 & \href{https://mangaki.fr/anime/9358}{Carnival Phanta

In [71]:
get_ratings(Work.objects.filter(title__icontains='Fujiko')[3])

Counter({'dislike': 1, 'like': 14, 'neutral': 5, 'willsee': 5, 'wontsee': 4})

In [43]:
Work.objects.order_by('-nb_dislikes').values_list('title', 'nb_dislikes')

<WorkQuerySet [('Fairy Tail', 238), ('Naruto: Shippuuden', 226), ('Naruto', 211), ('Bleach', 178), ('Dragon Ball GT', 161), ('Yu-Gi-Oh! GX', 158), ('School Days', 155), ('Sword Art Online', 136), ('One Piece', 117), ('Dragon Ball Z Movie 11: Bio-Broly', 107), ('Kiss x Sis', 96), ('Rosario to Vampire', 96), ('Highschool of the Dead', 94), ('IS: Infinite Stratos', 92), ('Naruto the Movie 2: Legend of the Stone of Gelel', 89), ('Fairy Tail', 88), ('Vampire Knight', 82), ('Dragon Ball Z', 80), ('Angel Beats!', 74), ('Sword Art Online II', 73), '...(remaining elements truncated)...']>

<WorkQuerySet [<Work: Fullmetal Alchemist - Edition reliée>, <Work: FullMetal Alchemist>, <Work: Fullmetal Alchemist: Brotherhood>, <Work: Steins;Gate>, <Work: Princesse Mononoké>, <Work: Le Voyage de Chihiro>, <Work: Tokyo ghoul>, <Work: L'Attaque des Titans>, <Work: Death note>, <Work: Your Lie in April>]>

In [31]:
for work in Work.objects.top().reverse()[:10]:
    print(work.title, work.nb_likes, 'likes', work.nb_dislikes, 'dislikes')

Girls Bravo: First Season 26 likes 39 dislikes
Astarotte's Toy 31 likes 38 dislikes
Captain Earth 25 likes 35 dislikes
Vividred Operation 13 likes 22 dislikes
Choujigen Game Neptune: The Animation 27 likes 33 dislikes
Ai non stop! 24 likes 33 dislikes
Haruka Nogizaka's Secret 24 likes 27 dislikes
Dragon Ball GT 87 likes 161 dislikes
Dog Days 55 likes 66 dislikes
Glasslip 23 likes 36 dislikes
